In [3]:
from datetime import datetime
import os
import random
import matplotlib
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#import seaborn as sns
#sns.set_style('whitegrid')

from scipy import sparse
from scipy.sparse import csc_matrix

from sklearn.decomposition import TruncatedSVD
#from sklearn.metrics.pariwise import cosine_similarity

In [4]:
print('creating the dataframe from data.csv file..')
#df = pd.read_csv('data.csv', sep=',', names=['movie','user','rating','date'])
#!wget https://storage.cloud.google.com/cs-552-final-project/netflix-data/Netflix_All_Data.parquet
df = pd.read_parquet('netflix_data_Netflix_All_Data.parquet')
df.date = pd.to_datetime(df.date)
print('Done.\n')

#arranging the rating according to time
print('sorting the dataframe by date..')
df.sort_values(by='date', inplace=True)
print('sorting done.')

creating the dataframe from data.csv file..
Done.

sorting the dataframe by date..
sorting done.


In [3]:
df.head()

,movie,user,rating,date
0,9798,510180,3,1999-11-11
17,8651,510180,2,1999-11-11
16,5571,510180,4,1999-11-11
15,3870,510180,2,1999-11-11
14,8079,510180,2,1999-11-11


In [5]:
df.describe()

,movie,user,rating
count,6.666330e+07,6.666330e+07,6.666330e+07
mean,6.085573e+03,1.322431e+06,3.595957e+00
std,3.451788e+03,7.645386e+05,1.087503e+00
min,1.000000e+00,6.000000e+00,1.000000e+00
25%,3.239000e+03,6.611680e+05,3.000000e+00
50%,6.037000e+03,1.318904e+06,4.000000e+00
75%,9.016000e+03,1.984378e+06,4.000000e+00
max,1.217100e+04,2.649429e+06,5.000000e+00


In [6]:
df.describe()['rating']

count    6.666330e+07
mean     3.595957e+00
std      1.087503e+00
min      1.000000e+00
25%      3.000000e+00
50%      4.000000e+00
75%      4.000000e+00
max      5.000000e+00
Name: rating, dtype: float64

**Checking NaN values**

In [7]:
print('number of NaN values in our dataset:', sum(df.isnull().any()))

number of NaN values in our dataset: 0


Check and Remove Duplicate

In [8]:
dup = df.duplicated(['movie','user','rating'])
dups = sum(dup) #considering by column
print('there are {} duplicate rating entries in the data.....'.format(dups))

there are 0 duplicate rating entries in the data.....


Basic Stats

In [1]:
import torch
from torch.autograd import Variable

#matrix factorization
class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors,
                                               sparse=True)
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors,
                                               sparse=True)

    def forward(self, user, item):
        # matrix multiplication
        return (self.user_factors(user)*self.item_factors(item)).sum(1)

In [5]:
print('Total Data')
print("-"*60)
print('\nTotal number of rating:', df.shape[0])
print('Total number of users:', len(np.unique(df.user)))
print('total number of movie:', len(np.unique(df.movie)))

Total Data
------------------------------------------------------------

Total number of rating: 66663300
Total number of users: 479101
total number of movie: 12171


In [6]:
new_data=df[['movie','user','rating']]
new_data

,movie,user,rating
0,9798,510180,3
17,8651,510180,2
16,5571,510180,4
15,3870,510180,2
14,8079,510180,2
...,...,...,...
66644331,2913,1270613,2
66644330,11315,607160,5
66644329,9120,692705,3
66644340,8387,1146063,4


In [7]:
_50_movies = new_data['movie'].value_counts()[0:50].reset_index()['index'].values

In [8]:
partial_data = new_data[new_data['movie'].isin(_50_movies)]

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
from scipy import sparse
from scipy.sparse import csr_matrix
train_df,test_df = train_test_split(partial_data,test_size=0.2)
print(train_df.shape)
sample_train_df = train_df.sample(frac=0.1)
mat = sparse.coo_matrix((sample_train_df["rating"], (sample_train_df["user"], sample_train_df["movie"])))

(5866555, 3)


In [11]:
matrix = sparse.csc_matrix(mat)

In [12]:
rows, cols = mat.nonzero()
p = np.random.permutation(len(rows))
rows, cols = rows[p], cols[p]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
torch.cuda.is_available()

False

In [14]:
#train for matrix factorization
def train(n_factors):
    model = MatrixFactorization(matrix.shape[0], matrix.shape[1], n_factors)
    loss_func = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=1e-6)
    count = 0
    for row, col in zip(*(rows, cols)):
        # Set gradients to zero
        count += 1
        optimizer.zero_grad()

        # Turn data into tensors
        rating = torch.FloatTensor([matrix[row, col]])
        row = torch.LongTensor([row])
        col = torch.LongTensor([col])

        # Predict and calculate loss
        prediction = model(row, col)
        loss = loss_func(prediction, rating)

        # Backpropagate
        loss.backward()

        # Update the parameters
        optimizer.step()
        if count % 10000 == 0:
            print('progress: ' + str(count/1000) + 'k')
    return model

In [15]:
#testing data
sample_test_df = test_df.sample(frac=0.1)
sample_test_mat = sparse.coo_matrix((sample_test_df["rating"], (sample_test_df["user"], sample_test_df["movie"])))
matrix = csc_matrix(mat)
rows, cols = matrix.nonzero()

In [16]:
import torch
def test(model):
    optimizer = torch.optim.SGD(model.parameters(),lr=1e-6)
    from sklearn.preprocessing import MinMaxScaler
    import numpy as np
    count = 0
    pred_list = []
    rating_list = []
    for row, col in zip(*(rows, cols)):
        # Set gradients to zero
        count += 1
        optimizer.zero_grad()

        # Turn data into tensors
        rating = matrix[row, col]
        row = torch.LongTensor([row])
        col = torch.LongTensor([col])

        # Predict and calculate loss
        prediction = model(row, col)
        pred_rating = prediction[0].item()


        print('pred: ' + str(pred_rating) + ' rating: ' + str(rating))
        pred_list.append(pred_rating)
        rating_list.append(rating)

        if count % 10000 == 0:
            print('progress: ' + str(count/1000) + 'k')
        if count == 100:
            break
    scaler = MinMaxScaler()

    print(1)
    pred_list = np.array(pred_list).reshape(-1,1)


    scaler.fit(pred_list)
    normalized_pred_list = scaler.transform(pred_list)*5

    count = 0
    for x, y in zip(normalized_pred_list, rating_list):
        if abs(x-y) <=1:
            count += 1
        print('pred: ' + str(x[0]) + ' rating: ' + str(y))
    return count/len(rating_list)

In [17]:
#TODO hyper parameter
result_list = []
for i in range(20,51):
    model = train(i)
    test_result = test(model)
    result_list.append((i, test_result))
    print(str(i)+': '+str(test_result))

progress: 10.0k
progress: 20.0k
progress: 30.0k
progress: 40.0k
progress: 50.0k
progress: 60.0k
progress: 70.0k
progress: 80.0k
progress: 90.0k
progress: 100.0k
progress: 110.0k
progress: 120.0k
progress: 130.0k
progress: 140.0k
progress: 150.0k
progress: 160.0k
progress: 170.0k
progress: 180.0k
progress: 190.0k
progress: 200.0k
progress: 210.0k
progress: 220.0k
progress: 230.0k
progress: 240.0k
progress: 250.0k
progress: 260.0k
progress: 270.0k
progress: 280.0k
progress: 290.0k
progress: 300.0k
progress: 310.0k
progress: 320.0k
progress: 330.0k
progress: 340.0k
progress: 350.0k
progress: 360.0k
progress: 370.0k
progress: 380.0k
progress: 390.0k
progress: 400.0k
progress: 410.0k
progress: 420.0k
progress: 430.0k
progress: 440.0k
progress: 450.0k
progress: 460.0k
progress: 470.0k
progress: 480.0k
progress: 490.0k
progress: 500.0k
progress: 510.0k
progress: 520.0k
progress: 530.0k
progress: 540.0k
progress: 550.0k
progress: 560.0k
progress: 570.0k
progress: 580.0k
pred: -10.6377754211425

In [18]:
print(result_list)

[(20, 0.24), (21, 0.36), (22, 0.42), (23, 0.42), (24, 0.36), (25, 0.46), (26, 0.35), (27, 0.35), (28, 0.33), (29, 0.34), (30, 0.49), (31, 0.39), (32, 0.22), (33, 0.4), (34, 0.41), (35, 0.32), (36, 0.43), (37, 0.42), (38, 0.33), (39, 0.32), (40, 0.35), (41, 0.32), (42, 0.25), (43, 0.38), (44, 0.41), (45, 0.37), (46, 0.26), (47, 0.23), (48, 0.31), (49, 0.48), (50, 0.21)]


In [19]:
#Dense Net
from torch import nn
import torch.nn.functional as F
class DenseNet(nn.Module):

    def __init__(self, n_users, n_items, n_factors, H1=128, D_out=1):
        super().__init__()
        self.user_factors = torch.nn.Embedding(n_users, n_factors, sparse=True)
        self.item_factors = torch.nn.Embedding(n_items, n_factors, sparse=True)
        self.l1 = torch.nn.Linear(n_factors*2, H1)
        self.l2 = torch.nn.Linear(H1, D_out)

    def forward(self, users, items):
        users_em = self.user_factors(users)
        items_em = self.item_factors(items)
        x = torch.cat([users_em, items_em], 1)
        h1_relu = F.relu(self.l1(x))
        output = self.l2(h1_relu)
        return output

    def predict(self, users, items):
        return self.forward(users, items)

In [20]:
#train for densenet
def train2(n_factors):
    model = DenseNet(matrix.shape[0], matrix.shape[1], n_factors, 128, 1)
    loss_func = torch.nn.MSELoss()
    optimizer = torch.optim.SGD(model.parameters(),lr=1e-6)
    count = 0
    for row, col in zip(*(rows, cols)):
        # Set gradients to zero
        count += 1
        optimizer.zero_grad()

        # Turn data into tensors
        rating = torch.FloatTensor([matrix[row, col]])
        row = torch.LongTensor([row])
        col = torch.LongTensor([col])

        # Predict and calculate loss
        prediction = model(row, col)
        loss = loss_func(prediction, rating)

        # Backpropagate
        loss.backward()

        # Update the parameters
        optimizer.step()
        if count % 10000 == 0:
            print('progress: ' + str(count/1000) + 'k')
    return model

In [21]:
#TODO hyper parameter
result_list = []
for i in range(20,51):
    model = train2(i)
    test_result = test(model)
    result_list.append((i, test_result))
    print(str(i)+': '+str(test_result))

/home/ning/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


progress: 10.0k
progress: 20.0k
progress: 30.0k
progress: 40.0k
progress: 50.0k
progress: 60.0k
progress: 70.0k
progress: 80.0k
progress: 90.0k
progress: 100.0k
progress: 110.0k
progress: 120.0k
progress: 130.0k
progress: 140.0k
progress: 150.0k
progress: 160.0k
progress: 170.0k
progress: 180.0k
progress: 190.0k
progress: 200.0k
progress: 210.0k
progress: 220.0k
progress: 230.0k
progress: 240.0k
progress: 250.0k
progress: 260.0k
progress: 270.0k
progress: 280.0k
progress: 290.0k
progress: 300.0k
progress: 310.0k
progress: 320.0k
progress: 330.0k
progress: 340.0k
progress: 350.0k
progress: 360.0k
progress: 370.0k
progress: 380.0k
progress: 390.0k
progress: 400.0k
progress: 410.0k
progress: 420.0k
progress: 430.0k
progress: 440.0k
progress: 450.0k
progress: 460.0k
progress: 470.0k
progress: 480.0k
progress: 490.0k
progress: 500.0k
progress: 510.0k
progress: 520.0k
progress: 530.0k
progress: 540.0k
progress: 550.0k
progress: 560.0k
progress: 570.0k
progress: 580.0k
pred: 4.069460868835449

/home/ning/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


progress: 10.0k
progress: 20.0k
progress: 30.0k
progress: 40.0k
progress: 50.0k
progress: 60.0k
progress: 70.0k
progress: 80.0k
progress: 90.0k
progress: 100.0k
progress: 110.0k
progress: 120.0k
progress: 130.0k
progress: 140.0k
progress: 150.0k
progress: 160.0k
progress: 170.0k
progress: 180.0k
progress: 190.0k
progress: 200.0k
progress: 210.0k
progress: 220.0k
progress: 230.0k
progress: 240.0k
progress: 250.0k
progress: 260.0k
progress: 270.0k
progress: 280.0k
progress: 290.0k
progress: 300.0k
progress: 310.0k
progress: 320.0k
progress: 330.0k
progress: 340.0k
progress: 350.0k
progress: 360.0k
progress: 370.0k
progress: 380.0k
progress: 390.0k
progress: 400.0k
progress: 410.0k
progress: 420.0k
progress: 430.0k
progress: 440.0k
progress: 450.0k
progress: 460.0k
progress: 470.0k
progress: 480.0k
progress: 490.0k
progress: 500.0k
progress: 510.0k
progress: 520.0k
progress: 530.0k
progress: 540.0k
progress: 550.0k
progress: 560.0k
progress: 570.0k
progress: 580.0k
pred: 4.963992595672607

/home/ning/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

/home/ning/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


progress: 10.0k
progress: 20.0k
progress: 30.0k
progress: 40.0k
progress: 50.0k
progress: 60.0k
progress: 70.0k
progress: 80.0k
progress: 90.0k
progress: 100.0k
progress: 110.0k
progress: 120.0k
progress: 130.0k
progress: 140.0k
progress: 150.0k
progress: 160.0k
progress: 170.0k
progress: 180.0k
progress: 190.0k
progress: 200.0k
progress: 210.0k
progress: 220.0k
progress: 230.0k
progress: 240.0k
progress: 250.0k
progress: 260.0k
progress: 270.0k
progress: 280.0k
progress: 290.0k
progress: 300.0k
progress: 310.0k
progress: 320.0k
progress: 330.0k
progress: 340.0k
progress: 350.0k
progress: 360.0k
progress: 370.0k
progress: 380.0k
progress: 390.0k
progress: 400.0k
progress: 410.0k
progress: 420.0k
progress: 430.0k
progress: 440.0k
progress: 450.0k
progress: 460.0k
progress: 470.0k
progress: 480.0k
progress: 490.0k
progress: 500.0k
progress: 510.0k
progress: 520.0k
progress: 530.0k
progress: 540.0k
progress: 550.0k
progress: 560.0k
progress: 570.0k
progress: 580.0k
pred: 3.82024884223938 

/home/ning/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


progress: 10.0k
progress: 20.0k
progress: 30.0k
progress: 40.0k
progress: 50.0k
progress: 60.0k
progress: 70.0k
progress: 80.0k
progress: 90.0k
progress: 100.0k
progress: 110.0k
progress: 120.0k
progress: 130.0k
progress: 140.0k
progress: 150.0k
progress: 160.0k
progress: 170.0k
progress: 180.0k
progress: 190.0k
progress: 200.0k
progress: 210.0k
progress: 220.0k
progress: 230.0k
progress: 240.0k
progress: 250.0k
progress: 260.0k
progress: 270.0k
progress: 280.0k
progress: 290.0k
progress: 300.0k
progress: 310.0k
progress: 320.0k
progress: 330.0k
progress: 340.0k
progress: 350.0k
progress: 360.0k
progress: 370.0k
progress: 380.0k
progress: 390.0k
progress: 400.0k


KeyboardInterrupt: 